In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from scipy.interpolate import griddata

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

# 1. 시군 설정 및 데이터 경로 매핑
regions = {
    "이천시": ("data/체감온도/이천시_WBGT.csv", "data/행정동경계_원천데이터/경기도/LARD_ADM_SECT_SGG_41_202505.shp"),
    "구리시": ("data/체감온도/구리시_WBGT.csv", "data/행정동경계_원천데이터/경기도/LARD_ADM_SECT_SGG_41_202505.shp"),
    "세종시": ("data/체감온도/세종시_WBGT.csv", "data/행정동경계_원천데이터/세종특별자치시/LARD_ADM_SECT_SGG_36_202505.shp"),
}

# 2. 전체 WBGT 범위 계산 (vmin, vmax)
all_wbgt = []
for csv_path, _ in regions.values():
    df = pd.read_csv(csv_path)
    all_wbgt.extend(df["WBGT"].values)

vmin = np.floor(min(all_wbgt) * 10) / 10
vmax = np.ceil(max(all_wbgt) * 10) / 10

# 3. subplot 생성
fig, axes = plt.subplots(1, 3, figsize=(18, 6), constrained_layout=True)
fig.suptitle("시군별 WBGT 연평균 분포도", fontsize=16)

# 4. 각 subplot에 시각화
for ax, (city, (csv_path, shp_path)) in zip(axes, regions.items()):
    df = pd.read_csv(csv_path)
    lon, lat, wbgt = df["lon"], df["lat"], df["WBGT"]

    gdf = gpd.read_file(shp_path).to_crs("EPSG:4326")

    # 경계 추출 (세종시는 도시 이름 필터 생략)
    if city == "세종시":
        boundary = gdf.geometry.unary_union
    else:
        boundary = gdf[gdf["SGG_NM"].str.contains(city)].geometry.unary_union


    # 그리드 및 보간
    grid_lon = np.linspace(lon.min(), lon.max(), 300)
    grid_lat = np.linspace(lat.min(), lat.max(), 300)
    grid_x, grid_y = np.meshgrid(grid_lon, grid_lat)
    grid_z = griddata((lon, lat), wbgt, (grid_x, grid_y), method="cubic")

    # 마스킹
    mask = np.array([[boundary.contains(Point(x, y)) for x in grid_lon] for y in grid_lat])
    masked_z = np.where(mask, grid_z, np.nan)

    # 시각화
    pcm = ax.pcolormesh(grid_x, grid_y, masked_z, cmap="YlOrRd", shading="auto")
    gpd.GeoSeries(boundary).boundary.plot(ax=ax, edgecolor="black", linewidth=1)

    ax.set_title(city)
    ax.set_xlabel("경도")
    ax.set_ylabel("위도")
    ax.set_aspect("equal")

# 5. 공통 colorbar 추가
cbar = fig.colorbar(pcm, ax=axes.ravel().tolist(), shrink=0.8, orientation="vertical", label="WBGT (°C)")

plt.show()

C:\Users\dodam\AppData\Local\Temp\ipykernel_26044\1895652398.py:42: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  boundary = gdf[gdf["SGG_NM"].str.contains(city)].geometry.unary_union
C:\Users\dodam\AppData\Local\Temp\ipykernel_26044\1895652398.py:42: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  boundary = gdf[gdf["SGG_NM"].str.contains(city)].geometry.unary_union
C:\Users\dodam\AppData\Local\Temp\ipykernel_26044\1895652398.py:40: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  boundary = gdf.geometry.unary_union
